In [1]:
# Step 1: Mount Google Drive (optional if you want to save the model there)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import shutil

gdrive_path = "/content/drive/My Drive/hough_segmented_dataset.zip"
local_path = "hough_segmented_dataset.zip"

# Copy zip to local environment for faster access
shutil.copy(gdrive_path, local_path)



'hough_segmented_dataset.zip'

In [3]:
# Step 3: Unzip dataset
import zipfile

with zipfile.ZipFile(local_path, 'r') as zip_ref:
    zip_ref.extractall("data")

dataset_dir = "data/hough_segmented_dataset"


In [ ]:
import os
import shutil

def reorganize_by_front_back(base_dir):
    for coin_class in os.listdir(base_dir):
        class_path = os.path.join(base_dir, coin_class)
        if not os.path.isdir(class_path):
            continue

        # Create front and back subfolders
        front_dir = os.path.join(class_path, "front")
        back_dir = os.path.join(class_path, "back")
        os.makedirs(front_dir, exist_ok=True)
        os.makedirs(back_dir, exist_ok=True)

        for filename in os.listdir(class_path):
            file_path = os.path.join(class_path, filename)
            if os.path.isfile(file_path):
                lowercase_name = filename.lower()
                if "front" in lowercase_name:
                    shutil.move(file_path, os.path.join(front_dir, filename))
                elif "back" in lowercase_name:
                    shutil.move(file_path, os.path.join(back_dir, filename))

# Apply to dataset directory
reorganize_by_front_back(dataset_dir)


In [4]:
# Step 4: Create image dataset from folder
import tensorflow as tf

# Flatten subfolders (front/back) under each class
def flatten_and_reorganize(dataset_dir):
    import shutil
    import os
    import glob

    flat_dir = "flat_dataset"
    os.makedirs(flat_dir, exist_ok=True)

    for coin_class in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, coin_class)
        if os.path.isdir(class_path):
            for view in ["front", "back"]:
                view_path = os.path.join(class_path, view)
                if os.path.exists(view_path):
                    target_class_dir = os.path.join(flat_dir, coin_class)
                    os.makedirs(target_class_dir, exist_ok=True)
                    for img_file in glob.glob(os.path.join(view_path, "*")):
                        shutil.copy(img_file, target_class_dir)
    return flat_dir

flat_dataset_path = flatten_and_reorganize(dataset_dir)


In [5]:
# Step 5: Load the images as tf.data.Dataset
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    flat_dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    flat_dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


Found 4721 files belonging to 7 classes.
Using 3777 files for training.
Found 4721 files belonging to 7 classes.
Using 944 files for validation.
Classes: ['10c', '20c', '50c', '5c', 'R1', 'R2', 'R5']


In [6]:
# Step 6: Optimize dataset performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [7]:
# Step 7: Build a simple CNN
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(*IMG_SIZE, 3)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,630,247 (6.22 MB)

 Trainable params: 1,630,247 (6.22 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Step 8: Train the model
EPOCHS = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)


Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.3721 - loss: 1.6132 - val_accuracy: 0.5964 - val_loss: 1.1100
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6277 - loss: 0.9938 - val_accuracy: 0.6663 - val_loss: 0.9462
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7357 - loss: 0.7481 - val_accuracy: 0.7097 - val_loss: 0.8403
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8012 - loss: 0.5707 - val_accuracy: 0.7415 - val_loss: 0.7486
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8730 - loss: 0.3796 - val_accuracy: 0.7765 - val_loss: 0.6792
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9176 - loss: 0.2620 - val_accuracy: 0.7765 - val_loss: 0.7719
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9297 - loss: 0.2353 - val_accuracy: 0.7913 - val_loss: 0.7335
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9434 - loss: 0.1875 - val_accuracy: 

In [9]:
# Step 9: Convert to TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("real_time_coin_detector(hough).tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmplu9itbir'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  139212787021968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212787022736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212787022160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212780897488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212780898256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212780896912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212780898448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212780897104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212780899792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139212780900752: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# Step 10: Download the TFLite model
from google.colab import files
files.download("real_time_coin_detector(hough).tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>